![Cloud-First](../image/CloudFirst.png) 


# SIT742: Modern Data Science 
**(Module: Big Data Analytics)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, change and distribute this package.
- If you found any issue/bug for this document, please submit an issue at [tulip-lab/sit742](https://github.com/tulip-lab/sit742/issues)


Prepared by **SIT742 Teaching Team**

---


## Session 5B: Arima Models

**The purpose of this session is to illustrate**

1. How to use Arima for Time Series Prediction.

** References and additional reading and resources**
-[Another ARIMA Tutorial](https://machinelearningmastery.com/arima-for-time-series-forecasting-with-python/) 


## ARIMA Time Series Forecasting

**A time series is a sequence where a metric is recorded over regular time intervals. Forecasting on time series is the next step where you want to predict the future values the series is going to take. If you use only the previous values of the time series to predict its future values, it is called Univariate Time Series Forecasting.
And if you use predictors other than the series (a.k.a exogenous variables) to forecast it is called Multi Variate Time Series Forecasting. In this notebook, we focuses on a particular type of forecasting method called ARIMA**

To do:

1. Understand the Arima model with parameter of p, d, q.
2. Understand how to select p, d and q.
3. How to use Arima for Time Series Prediction.

---

### Task1 Introduction to ARIMA Models with parameter of p, d and q

ARIMA, short for ‘Auto Regressive Integrated Moving Average’ is actually a class of models that ‘explains’ a given time series based on its own past values, that is, its own lags and the lagged forecast errors, so that equation can be used to forecast future values.

An ARIMA model is characterized by 3 terms: p, d, q

where,

p is the order of the AR term

q is the order of the MA term

d is the number of differencing required to make the time series stationary

The first thing to build ARIMA is to make the time series stationary, why?
Because, term ‘Auto Regressive’ in ARIMA means it is a linear regression model that uses its own lags as predictors. Linear regression models, as you know, work best when the predictors are not correlated and are independent of each other.

How to make it stationary?
The most common approach is to difference it. That is, subtract the previous value from the current value. Sometimes, depending on the complexity of the series, more than one differencing may be needed. That is why we need parameter 'd' here.

Next, what are the ‘p’ and ‘q’ terms?

'p' is the order of the ‘Auto Regressive’ (AR) term. It refers to the number of lags of Y to be used as predictors. And ‘q’ is the order of the ‘Moving Average’ (MA) term. It refers to the number of lagged forecast errors that should go into the ARIMA Model.

##### Resource for further understanding: please refer the cloudfirst M05B

### Task2 How to find the d, p ,q in ARIMA model

#### Task 2.1 Find the differencing paramter d

The right order of differencing is the minimum differencing required to get a near-stationary series which roams around a defined mean and the ACF plot reaches to zero fairly quick. A simple trick is:
If the autocorrelations are positive for many number of lags (10 or more), then the series needs further differencing. On the other hand, if the lag 1 autocorrelation itself is too negative, then the series is probably over-differenced.

Let's try it together.

In [ ]:
!pip install "statsmodels==0.11.1"

In [ ]:
import numpy as np, pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize':(20,12), 'figure.dpi':60})

# Import data
df = pd.read_csv('https://github.com/tulip-lab/sit742/raw/master/Jupyter/data/timeseries-data.txt', header=0)

# Original Series
fig, axes = plt.subplots(3, 2)
axes[0, 0].plot(df.Births); axes[0, 0].set_title('Original Series')
plot_acf(df.Births, ax=axes[0, 1])

# 1st Differencing
axes[1, 0].plot(df.Births.diff()); axes[1, 0].set_title('1st Order Differencing')
plot_acf(df.Births.diff().dropna(), ax=axes[1, 1])

# 2nd Differencing
axes[2, 0].plot(df.Births.diff().diff()); axes[2, 0].set_title('2nd Order Differencing')
plot_acf(df.Births.diff().diff().dropna(), ax=axes[2, 1])

plt.show()

For the above series, the time series reaches stationarity with **0 order** (original) of differencing. But on looking at the autocorrelation plot for the 1st and 2nd differencing the lag goes into the far negative zone fairly quick, which indicates, the series might have been over differenced.

#### Task 2.2 Find the order of the AR term (p)

The next step is to identify if the model needs any AR terms. You can find out the required number of AR terms by inspecting the Partial Autocorrelation (PACF) plot.

Partial autocorrelation can be imagined as the correlation between the series and its lag, after excluding the contributions from the intermediate lags. So, PACF sort of conveys the pure correlation between a lag and the series. That way, you will know whether that lag is needed in the AR term or not.

Now, how to find the number of AR terms?

Any autocorrelation in a stationarized series can be rectified by adding enough AR terms. So, we initially take the order of AR term to be equal to as many lags that crosses the significance limit in the PACF plot.

In [ ]:
# PACF plot of 1st differenced series
plt.rcParams.update({'figure.figsize':(20,5), 'figure.dpi':120})

fig, axes = plt.subplots(2, 1)
axes[0].plot(df.Births); axes[0].set_title('No Differencing')
axes[1].set(ylim=(0,5))
plot_pacf(df.Births, ax=axes[1])

plt.show()

You can observe that the PACF lag 1-3 are all quite significant since they are well above the significance line.  But I am going to be conservative and tentatively fix the p as 1.

#### Task 2.3 Find the order of the MA term (q)

You can look at the ACF plot for the number of MA terms. An MA term is technically, the error of the lagged forecast.
The ACF tells how many MA terms are required to remove any autocorrelation in the stationarized series

In [ ]:
fig, axes = plt.subplots(2, 1)
axes[0].plot(df.Births); axes[0].set_title('No Differencing')
axes[1].set(ylim=(0,1.2))
plot_acf(df.Births, ax=axes[1])

plt.show()

Couple of lags are well above the significance line. So, let’s tentatively fix q as 1. When in doubt, go with the simpler model that sufficiently explains the Y

### Task 3 Build the ARIMA Model

Now that you’ve determined the values of p, d and q, you have everything needed to fit the ARIMA model. Let’s use the ARIMA() implementation in statsmodels package. 

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

# 1,0,1 ARIMA Model
model = ARIMA(df.Births, order=(1,0,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())

Look at the coef for ar.L1 and ma.L1, the p value is quite good, so it means the choice of p / q we choose is good.

Let’s plot the residuals to ensure there are no patterns (that is, look for constant mean and variance).

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()

The residual errors seem fine with near zero mean and uniform variance. Let’s plot the actuals against the fitted values using plot_predict().

## Optional (Programmatic Way of Using Arima with Grid Search)

Another way to conduct the Arima forecast is via gridsearch. So the pamameter is not manaully selected but through the 
grid search. This is quite popular when the training system need to be updated frequently in the practical case such as 
retailed industry.

In [ ]:
from pandas import read_csv
from matplotlib import pyplot

data ="https://github.com/tulip-lab/sit742/raw/master/Jupyter/data/timeseries-data.txt"
series = read_csv(data, header=0, index_col=0)
series.plot(rot=45)
pyplot.show()

### Fit the series with Arima model and print out the prediction / conficence interval

In [ ]:
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA

# load dataset
series = read_csv(data, header=0, index_col=0, parse_dates=True, squeeze=True)

# split into train and test sets
X = series.values
X = X.astype('float32')
size = len(X) - 1
train, test = X[0:size], X[size:]

# fit an ARIMA model
model = ARIMA(train, order=(5,1,1))
model_fit = model.fit()
# forecast
result = model_fit.get_forecast()

# summarize forecast and confidence intervals
print('Expected: %.3f' % result.predicted_mean)
print('Forecast: %.3f' % test[0])
print('Standard Error: %.3f' % result.se_mean)
ci = result.conf_int(0.05)
print('95%% Interval: %.3f to %.3f' % (ci[0,0], ci[0,1]))

### Predict the time series by using walk-forward validation and Print out the RMSE

In [ ]:
# evaluate an ARIMA model using a walk-forward validation
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

series = read_csv(data, header=0, index_col=0, parse_dates=True, squeeze=True)

# split into train and test sets
X = series.values
X = X.astype('float32')
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:]
history = [x for x in train]
predictions = list()

# walk-forward validation
for t in range(len(test)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))

# evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

# plot forecasts against actual outcomes
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

### Conduct the Grid Search On Parameters

In [ ]:
# evaluate an ARIMA model using a walk-forward validation
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

series = read_csv(data, header=0, index_col=0, parse_dates=True, squeeze=True)

# split into train and test sets
X = series.values
X = X.astype('float32')
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:]
history = [x for x in train]
predictions = list()
RSME = []
p=[1,2,3]
q=[0,1]
d=[1,2]

# walk-forward validation
for i1 in p:
  for i2 in q:
    for i3 in d:
      for t in range(len(test)):
        model = ARIMA(history, order=(i1,i3,i2))
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test[t]
        history.append(obs)
  #print('predicted=%f, expected=%f' % (yhat, obs))  
      rmse = sqrt(mean_squared_error(test, predictions))
      history = [x for x in train]
      predictions = list()  
      RSME.append(rmse)
      print('Test RMSE: %.3f' % rmse,i1,i2,i3)   



### Using Best Paramter to perform train and also the walk-forward test

In [ ]:
# evaluate an ARIMA model using a walk-forward validation
import numpy as np
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
from math import sqrt

series = read_csv(data, header=0, index_col=0, parse_dates=True, squeeze=True)

# split into train and test sets
X = series.values
X = X.astype('float32')
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:]
history = [x for x in train]
predictions = list()
best_parameter = [2,1,1]
confidence_interval = []

# walk-forward validation
for t in range(len(test)):
    model = ARIMA(history, order=(best_parameter[0],best_parameter[1],best_parameter[2]))
    model_fit = model.fit()
    output = model_fit.get_forecast()
    yhat = output.predicted_mean
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print('predicted=%f, expected=%f' % (yhat, obs))
    ci = output.conf_int(0.05)
    confidence_interval.append(ci[0])
    print('95%% Interval: %.3f to %.3f' % (ci[0,0], ci[0,1]))
    
# evaluate forecasts
rmse = sqrt(mean_squared_error(test, predictions))
print('Test RMSE: %.3f' % rmse)

# plot forecasts against actual outcomes and also the confidence int at 95%
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.fill_between(list(range(len(test))),
                 np.array(confidence_interval)[:,0], np.array(confidence_interval)[:,1],
                alpha=0.1, color='b')
pyplot.show()